In [1]:
pip install farasapy

In [2]:
pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 29.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 15.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 13.

# **Importing Data**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd

In [5]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Classeur2.csv")

# **Tokenization**

Les textes sont divisés en mots à l'aide de la fonction word_tokenize du module NLTK, pour fragmenter chaque texte en éléments distincts (mots ou tokens).

In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

In [8]:
nltk.download('stopwords')
stop_words = set(stopwords.words('arabic'))
stemmer = SnowballStemmer('arabic')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
feed_text = df['Feed'].astype(str)
rating_int = df['Rating'].astype(int)

In [12]:
feed_tokens = feed_text.apply(word_tokenize)

# **Stemming**

Les tokens obtenus sont ensuite passés à travers un processus de racinisation (ou stemming) à l'aide de FarasaStemmer. Cela normalise les mots en les réduisant à leur racine, ce qui peut aider à regrouper les variations linguistiques d'un même mot.

In [13]:
from farasa.stemmer import FarasaStemmer
farasa_stemmer = FarasaStemmer(interactive=True)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'farasa-api.qcri.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


100%|██████████| 241M/241M [03:24<00:00, 1.18MiB/s]


[2023-12-28 18:24:56,886 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [14]:
def apply_stemming(tokens):
    stemmed_tokens = []
    for token in tokens:
        stemmed_token = farasa_stemmer.stem(token)
        stemmed_tokens.append(stemmed_token)
    return stemmed_tokens

In [15]:
feed_tokens = feed_tokens.apply(apply_stemming)

# **Nettoyage des données**

Les tokens vides ou blancs sont supprimés à l'aide de la fonction remove_blanks. Cela élimine les éléments vides ou composés uniquement d'espaces qui n'ont pas de signification dans l'analyse des textes.

In [16]:
def remove_blanks(tokens):
    return [token for token in tokens if token.strip()]


In [17]:
feed_tokens = feed_tokens.apply(remove_blanks)

In [18]:
combined_data = []
for idx in range(len(feed_tokens)):
    combined_data.append({
        'feed': ' '.join(feed_tokens[idx])
    })

In [19]:
feed_combined = []
for item in combined_data:
  feed_combined.append(item["feed"])

# **Embeddings**

Les embeddings de mots en arabe sont générés à l'aide de WordEmbeddings('ar'). Ces embeddings transforment chaque mot en une représentation vectorielle, permettant ainsi de capturer les significations et les relations entre les mots.
Les embeddings de documents sont créés à partir des embeddings de mots avec DocumentPoolEmbeddings. Ces embeddings agrègent les embeddings de mots de chaque phrase pour créer des représentations vectorielles des documents entiers.

In [20]:
from flair.embeddings import WordEmbeddings, DocumentRNNEmbeddings, DocumentPoolEmbeddings
from flair.data import Sentence
import numpy as np

In [21]:
word_embedding = WordEmbeddings('ar')
document_embeddings = DocumentPoolEmbeddings([word_embedding])

2023-12-28 18:33:09,497 https://flair.informatik.hu-berlin.de/resources/embeddings/token/ar-wiki-fasttext-300d-1M.vectors.npy not found in cache, downloading to /tmp/tmpwneiaic7


100%|██████████| 699M/699M [00:43<00:00, 17.0MB/s]

2023-12-28 18:33:54,546 copying /tmp/tmpwneiaic7 to cache at /root/.flair/embeddings/ar-wiki-fasttext-300d-1M.vectors.npy


2023-12-28 18:33:57,231 removing temp file /tmp/tmpwneiaic7
2023-12-28 18:33:58,163 https://flair.informatik.hu-berlin.de/resources/embeddings/token/ar-wiki-fasttext-300d-1M not found in cache, downloading to /tmp/tmpwppvuk5b


100%|██████████| 25.5M/25.5M [00:02<00:00, 12.9MB/s]

2023-12-28 18:34:00,720 copying /tmp/tmpwppvuk5b to cache at /root/.flair/embeddings/ar-wiki-fasttext-300d-1M
2023-12-28 18:34:00,757 removing temp file /tmp/tmpwppvuk5b


Les données textuelles subissent une transformation d'embedding :
Chaque texte est transformé en objet Sentence de Flair.
Les embeddings de documents sont calculés pour chaque phrase à l'aide de modèles pré-entraînés.
Les embeddings extraits sont convertis en tableaux numériques et stockés.

In [22]:
text_embeddings = []
for text in feed_text:
    sentence = Sentence(text)
    document_embeddings.embed(sentence)
    embedding = sentence.get_embedding().detach().cpu().numpy()
    text_embeddings.append(embedding)

text_embeddings = np.array(text_embeddings)

Les embeddings textuels obtenus sont combinés avec les valeurs de notation associées pour construire un ensemble de données d'entraînement prêt pour l'apprentissage automatique.
La concaténation des embeddings textuels avec les valeurs de notation prépare les données pour les modèles d'apprentissage automatique.

In [23]:
X = np.concatenate((text_embeddings, rating_int.values.reshape(-1, 1)), axis=1)


Les données d'entraînement et de test sont divisées à l'aide de train_test_split pour évaluer les performances du modèle. Les données d'entrée X contiennent les embeddings textuels concaténés avec les notations correspondantes.

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, rating_int, test_size=0.2, random_state=42)

# **Classification avec RandomForest **

Un classificateur RandomForest est choisi pour entraîner le modèle. Ce modèle est souvent utilisé pour des tâches de classification en machine learning.

In [26]:
from sklearn.ensemble import RandomForestClassifier


In [27]:
clf = RandomForestClassifier()


In [28]:
clf.fit(X_train, y_train)

RandomForestClassifier()

Le modèle est utilisé pour prédire les notations sur les données de test.
Les métriques d'évaluation telles que l'exactitude (Accuracy), la précision (Precision), le rappel (Recall), et le score F1 (F1 Score) sont calculées pour évaluer les performances du modèle sur les données de test.

In [29]:
predictions = clf.predict(X_test)

In [30]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [31]:
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions, average='weighted', zero_division=1)
recall = recall_score(y_test, predictions, average='weighted')
f1 = f1_score(y_test, predictions, average='weighted')

Le modèle entraîné montre les performances suivantes sur les données de test :
Exactitude (Accuracy) : Mesure de la justesse des prédictions.
Précision (Precision) : Indique la proportion de prédictions correctes parmi celles prédites comme positives.
Rappel (Recall) : Taux de détection des vrais positifs par rapport à tous les éléments positifs.
Score F1 (F1 Score) : Moyenne harmonique de la précision et du rappel, exprimant un équilibre entre les deux métriques.

In [32]:
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


Accuracy: 0.8878
Precision: 0.8904
Recall: 0.8878
F1 Score: 0.8650
